# The Merry Movie Montage as MinMax Colored Traveling Salesman Problem

This notebook is an attempt to use [LKH-3](http://webhotel4.ruc.dk/~keld/research/LKH-3/) CTSP solver with MinMax mTSP objective for this competition. It is based on [my previous notebook](https://www.kaggle.com/kostyaatarik/colored-traveling-salesman-problem) and brings two new ideas:
* edit LKH-3 source code to make CTSP solver use MinMax mTSP objective;
* provide complementary nodes to CTSP solver as fixed edges and get rid of the additional weight factor.

# What are the mTSP objectives?

Solving an mTSP instance one can strive various goals:
1. minimize the length of the overall tour;
2. minimize the size of the largest route;
3. minimize the length of the longest route.

The third variant is the best match for our problem and LKH-3 mTSP solver supports all these objectives.  
It's controlled by the parameter `MTSP_OBJECTIVE = [ MINSUM | MINMAX_SIZE | MINMAX ]` in the parameter file.

But unluckily LKH-3 CTSP solver doesn't take this parameter into account and its objective is `MINSUM`.

To change this behaviour we'll take four steps:
1. examine the LKH-3 CTSP penalty function's source code;
2. take a look at the LKH-3 mTSP MinMax penalty function;
3. edit CTSP penalty function to add MinMax objective to it;
4. compile modified code.

# What are the LKH penalty functions?

LKH tackles various special problems like mTSP and CTSP  by the means of penalties. Penalty is a special value associated with a solution but separate from the solution's score. Comparing two distinct solutions for the problem first of all LKH strives to minimize the penalty value and only if penalty values are the same solutions are judged by their scores.

# 1. CTSP Penalty Function

One can find this function in the file `LKH-3.0.7/SRC/Penalty_CTSP.c` after downloading and unpacking the [LKH-3 sources](http://webhotel4.ruc.dk/~keld/research/LKH-3/LKH-3.0.7.tgz). Let's have a look at it
```C
GainType Penalty_CTSP()
{
    static Node *StartRoute = 0;
    Node *N, *N1, *N2, *CurrentRoute;
    GainType P = 0;
    int Forward;

    N1 = Depot;
    while ((N1 = SUCC(N1))->DepotId == 0);
    N2 = Depot;
    while ((N2 = PREDD(N2))->DepotId == 0);
    Forward = N1 != N2 ? N1->DepotId < N2->DepotId : !Reversed;

    if (!StartRoute)
        StartRoute = Depot;
    N = StartRoute;
    do {
        CurrentRoute = N;
        do {
            if (N->Color != 0 && N->Color != CurrentRoute->DepotId) // penalty is
                P++;                                                // calculated here
        } while ((N = Forward ? SUCC(N) : PREDD(N))->DepotId == 0);
        if (P > CurrentPenalty ||
            (P == CurrentPenalty && CurrentGain <= 0)) {
            StartRoute = CurrentRoute;
            return CurrentPenalty + (CurrentGain > 0);
        }
    } while (N != StartRoute);
    return P;
}
```
The function is pretty straightforward, all it is doing is counting the number of colored nodes that ended up not in their own route. It won't be difficult to modify this code.

# 2. MTSP MinMax Penalty Function

This function can be found in the file `LKH-3.0.7/SRC/Penalty_MTSP.c`. Let's see it
```C
GainType Penalty_MTSP_MINMAX()
{
    int Forward = SUCC(Depot)->Id != Depot->Id + DimensionSaved;
    static Node *StartRoute = 0;
    Node *N, *NextN, *CurrentRoute;
    GainType Cost, MaxCost = MINUS_INFINITY;

    if (!StartRoute)
        StartRoute = Depot;
    if (StartRoute->Id > DimensionSaved)
        StartRoute -= DimensionSaved;
    N = StartRoute;
    do {
        Cost = 0;
        CurrentRoute = N;
        do {  // calculate the route's cost for the next salesman
            NextN = Forward ? SUCC(N) : PREDD(N);
            Cost += C(N, NextN) - N->Pi - NextN->Pi;
            if (NextN->Id > DimensionSaved)
                NextN = Forward ? SUCC(NextN) : PREDD(NextN);
        } while ((N = NextN)->DepotId == 0);
        Cost /= Precision;
        if (Cost > MaxCost) {  // update maximum cost
            if (Cost > CurrentPenalty ||
                (Cost == CurrentPenalty && CurrentGain <= 0)) {
                StartRoute = CurrentRoute;
                return CurrentPenalty + (CurrentGain > 0);
            }
            MaxCost = Cost;
        }
    } while (N != StartRoute);
    return MaxCost;  // return maximum cost
}
```
This penalty function is trickier but only a bit, it finds routes' costs for each salesman and returns the maximum of them. It will be easy to combine it with the previous penalty function.

# 3. CTSP MinMax Penalty Function

After adding MinMax objective to CTSP penalty function I ended up with the following code

```C
GainType Penalty_CTSP()
{
    static Node *StartRoute = 0;
    Node *N, *N1, *N2, *CurrentRoute, *NextN;
    GainType P = 0;
    GainType Cost, MaxCost = MINUS_INFINITY;
    int Forward;

    N1 = Depot;
    while ((N1 = SUCC(N1))->DepotId == 0);
    N2 = Depot;
    while ((N2 = PREDD(N2))->DepotId == 0);
    Forward = N1 != N2 ? N1->DepotId < N2->DepotId : !Reversed;

    if (!StartRoute)
        StartRoute = Depot;
    N = StartRoute;
    do {
        CurrentRoute = N;
        do {
            if (N->Color != 0 && N->Color != CurrentRoute->DepotId)
                P += 100000;  // valid CTSP solution is the priority over MinMax objective
        } while ((N = Forward ? SUCC(N) : PREDD(N))->DepotId == 0);
        if (P > CurrentPenalty ||
            (P == CurrentPenalty && CurrentGain <= 0)) {
            StartRoute = CurrentRoute;
            return CurrentPenalty + (CurrentGain > 0);
        }
        Cost = 0;
        N = CurrentRoute;
        do {
            NextN = Forward ? SUCC(N) : PREDD(N);
            Cost += C(N, NextN) - N->Pi - NextN->Pi;
            if (NextN->Id > DimensionSaved)
                NextN = Forward ? SUCC(NextN) : PREDD(NextN);
        } while ((N = NextN)->DepotId == 0);
        Cost /= Precision;
        if (Cost > MaxCost) {
            if (Cost > CurrentPenalty ||
                (Cost == CurrentPenalty && CurrentGain <= 0)) {
                StartRoute = CurrentRoute;
                return CurrentPenalty + (CurrentGain > 0);
            }
            MaxCost = Cost;
        }
    } while (N != StartRoute);
    return P + MaxCost;  // sum of CTSP and MinMax penalties
}
```
This code just literally calculates both CTSP penalty and MinMax penalty and returns their sum. But to prioritize the correctness of the solutions from the CTSP perspective over MinMax objective we changed the penalty value of one colored node being in the wrong route from \\(1\\) to \\(100000\\). If we'd left it being \\(1\\), LKH CTSP solver could mess mandatory permutations in strings up because it would be beneficial in terms of MinMax penalty.

# 4. Compile Modified Code

I uploaded a file with the modified version of the CTSP penalty function as a [dataset](https://www.kaggle.com/kostyaatarik/minmax-ctsp-penalty). So all we need to do is to download LKH-3 sources, unpack them, replace `LKH-3.0.7/SRC/Penalty_CTSP.c` with the modified version and than run make. 

In [ ]:
!wget http://webhotel4.ruc.dk/~keld/research/LKH-3/LKH-3.0.7.tgz &>/dev/null
!tar xvfz LKH-3.0.7.tgz &>/dev/null
!cp -r ../input/minmax-ctsp-penalty/Penalty_CTSP.c LKH-3.0.7/SRC/
!cd LKH-3.0.7; make &>/dev/null; cp LKH ..

# Fixed Edges

Transforming an asymmetric CTSP to a symmetric one in [my previous notebook](https://www.kaggle.com/kostyaatarik/colored-traveling-salesman-problem) we used additional weight factor DW to force edges between complementary nodes to be present in the output tour. In this notebook we'll provide complementary nodes to CTSP solver as fixed edges under the `FIXED_EDGES_SECTION` keyword, since additional weights would mess MinMax objective up.

In [ ]:
import functools
import glob
import itertools
import numpy as np
import pandas as pd

In [ ]:
SIZE = 5280 # total number of permutations
INF = 10**9 - 1 # infinite edge weight
TIME_LIMIT = 3600 * 2 # time limit for LKH run, seconds
SEED = 2021 # LKH seed value

def perm_dist(p, q):
    i = p.index(q[0])
    return i if p[i:] == q[:7-i] else 7

def distances_matrix():
    all_perms = list(itertools.permutations(range(7), 7))
    mandatory_perms = all_perms[:120]
    nodes = mandatory_perms * 2 + all_perms
    m = np.zeros((SIZE, SIZE), dtype='int32')
    for i, p in enumerate(nodes):
        for j, q in enumerate(nodes):
            m[i, j] = perm_dist(p, q)
    m *= 10
    m[np.where(m == 0)] = INF # treat equal perms in different mandatory sets
    np.fill_diagonal(m, 0) # restore zero weights at the main diagonal
    return m

def write_params_file(initial_tour=None):
    with open('santa.par', 'w') as f:
        printf = functools.partial(print, file=f)
        printf('SPECIAL')
        printf('PROBLEM_FILE = santa.ctsp')
        printf('TOUR_FILE = best_tour_$.txt') # $ will be replaced with the tour cost
        printf('OUTPUT_TOUR_FILE = output_tour_$.txt') # save each improvement
        if initial_tour:
            printf('INITIAL_TOUR_FILE = initial_tour.txt')
        printf('INITIAL_TOUR_ALGORITHM = CTSP')
        printf('MTSP_OBJECTIVE = MINMAX')
        printf('GAIN23 = YES')
        printf('PATCHING_C = 3')
        printf('PATCHING_A = 2')
        printf(f'SEED = {SEED}')
        printf('MAX_TRIALS = 100000')
        printf(f'TIME_LIMIT = {TIME_LIMIT}') # seconds
        printf('TRACE_LEVEL = 2')
        printf('PRECISION = 1')

def write_problem_file():
    with open('santa.ctsp', 'w', buffering=-1) as f:
        printf = functools.partial(print, file=f)
        printf('TYPE: CTSP')
        printf(f'DIMENSION: {SIZE * 2 + 1}')
        printf('SALESMEN : 3')
        printf('EDGE_WEIGHT_TYPE: EXPLICIT')
        printf('EDGE_WEIGHT_FORMAT: FULL_MATRIX')
        printf('EDGE_WEIGHT_SECTION')
        # write distances matrix
        inf_row = ' '.join(itertools.repeat(str(INF), SIZE))
        distances = distances_matrix()
        # top half of the distances matrix
        for weights in distances.T: # iterate over columns
            # infinite weights, weights column, distance to depot
            printf(inf_row, ' '.join(map(str, weights)), 35)
        # bottom half of the distances matrix
        for weights in distances: # iterate over rows
            # weights row, infinite weights, distance to depot
            printf(' '.join(map(str, weights)), inf_row, 35)
        printf(' '.join(itertools.repeat('35', SIZE * 2)), INF) # distances from the depot
        # write "private city sets"
        printf('CTSP_SET_SECTION')
        for i in range(3):
            printf(i + 1, end=' ') # set index 
            for j in range(1, 121):
                printf(i * 120 + j, end=' ') # real node of mandatory permutations
                printf(i * 120 + j + SIZE, end=' ') # complementary virtual node
            printf(-1)
        printf('FIXED_EDGES_SECTION')
        fixed_edges = zip(range(1, SIZE+1), range(SIZE+1, 2*SIZE+1))
        fixed_edges = itertools.chain.from_iterable(fixed_edges)
        printf(' '.join(map(str, fixed_edges)), -1)
        printf('DEPOT_SECTION')
        printf(2 * SIZE + 1)
        printf(-1)
        printf('EOF')

def write_initial_tour_file(initial_tour=None):
    if initial_tour:
        with open('initial_tour.txt', 'w') as f:
            print('TOUR_SECTION', file=f)
            print(' '.join(str(_) for _ in initial_tour), -1, file=f)
    
def solve_ctsp(initial_tour=None, verbose=False):
    write_params_file(initial_tour)
    write_problem_file()
    write_initial_tour_file(initial_tour)
    
    # run LKH-3 to solve CTSP instance
    if verbose:
        !./LKH santa.par
    else:
        !touch lkh.log
        !./LKH santa.par >> lkh.log

We'll provide LKH an initial tour to start optimization from. As the initial tour we'll use the one, found by [my previous notebook](https://www.kaggle.com/kostyaatarik/colored-traveling-salesman-problem), I uploaded it as a separate [dataset](https://www.kaggle.com/kostyaatarik/ctsp-v6-best-route).

In [ ]:
LETTERS = {
    1: '🎅',  # father christmas
    2: '🤶',  # mother christmas
    3: '🦌',  # reindeer
    4: '🧝',  # elf
    5: '🎄',  # christmas tree
    6: '🎁',  # gift
    7: '🎀',  # ribbon
    8: '🌟',  # star
}
INV_LETTERS = {v: k for k, v in LETTERS.items()}

solution = pd.read_csv('../input/santa-rebalancing-1/submission_no_wildcards_2456_2452_2442.csv')
strings = [[INV_LETTERS[c] for c in s] for s in solution.schedule]
strings.sort(key=len, reverse=True)
print(f'Strings lengths are {[len(_) for _ in strings]}.')

def find_strings_perms(strings, verbose=False):
    all_perms = set(itertools.permutations(range(1, 8), 7))
    perms = []
    for s in strings:
        perms.append([])
        for i in range(len(s)-6):
            p = tuple(s[i:i+7])
            if p in all_perms:
                perms[-1].append(p)
    if verbose:
        lens = [len(_) for _ in  perms]
        print(f'There are {lens} permutations in strings, {sum(lens)} in total.')
        lens = [len(set(_)) for _ in  perms]
        print(f'There are {lens} unique permutations in strings, {sum(lens)} in total.')
    return perms

def rebalance_perms(strings_perms, verbose=False):
    # convert to dicts for fast lookup and to keep permutations order
    strings_perms = [dict.fromkeys(_) for _ in strings_perms] 
    for p in strings_perms[0].copy():  # iterate over the copy to allow modification during iteration
        if p[:2] != (1, 2) and (p in strings_perms[1] or p in strings_perms[2]):
            strings_perms[0].pop(p)
    for p in strings_perms[1].copy():
        if p[:2] != (1, 2) and p in strings_perms[2]:
            strings_perms[1].pop(p)
    if verbose:
        lens = [len(_) for _ in  strings_perms]
        print(f'There are {lens} permutations left in strings after rebalancing, {sum(lens)} in total.')
    return [list(_) for _ in strings_perms]

strings_perms = find_strings_perms(strings, verbose=True)
strings_perms = rebalance_perms(strings_perms, verbose=True)

In [ ]:
def ctsp_initial_tour(strings_perms):
    index = {p: i for (i, p) in enumerate(itertools.permutations(range(1, 8), 7), 1)}
    initial_tour = []
    for i, perms in enumerate(strings_perms):
        initial_tour.append(SIZE*2 + i + 1) # depot node for each string
        for p in perms:
            if p[:2] == (1, 2):
                initial_tour.append(i*120 + index[p])
            else:
                initial_tour.append(240 + index[p])
            initial_tour.append(initial_tour[-1] + SIZE) # a complementary virtual node
    return initial_tour


initial_tour = ctsp_initial_tour(strings_perms)

Write all files and feed it to LKH.

In [ ]:
solve_ctsp(initial_tour)

Check all the improved tours found by LKH along the optimization.

In [ ]:
def read_strings(file_name):
    all_perms = list(itertools.permutations(range(1, 8), 7))
    mandatory_perms = all_perms[:120]
    nodes = mandatory_perms * 2 + all_perms
    
    with open(file_name, 'r') as f:
        lines = [l.strip() for l in f.readlines()]
    lines = lines[lines.index(f'{SIZE*2 + 1}'):-2]
    tour = [int(_) - 1 for _ in lines]
    i0, i1, i2 = sorted(tour.index(i) for i in range(SIZE*2, SIZE*2 + 3)) # depots
    strings = [tour[i0+1:i1], tour[i1+1:i2], tour[i2+1:]]
    for s in strings:
        s[:] = [nodes[_] for _ in s if _ < SIZE] # leave only real nodes
        s_forward, s_backward = [], []
        for directed_s in (s_forward, s_backward):
            directed_s.extend(s[0])
            for p, q in zip(s, s[1:]):
                d = perm_dist(p, q)
                directed_s.extend(q[-d:])
            s[:] = s[::-1]
        s[:] = min(s_forward, s_backward, key=len)
    return strings

def check_solution(strings):
    all_perms = set(itertools.permutations(range(1, 8), 7))
    mandatory_perms = {p for p in all_perms if p[:2] == (1, 2)}
    strings_perms = [set(_) for _ in find_strings_perms(strings)]
    for s in strings_perms:
        if mandatory_perms - s:
            print(mandatory_perms - s)
            return False
    if all_perms - set.union(*strings_perms):
        return False
    return True

def contain_wildcards(strings):
    for s in strings:
        if 8 in s:
            return True
    return False

def write_submission_csv(strings):
    sub = pd.DataFrame()
    sub['schedule'] = [''.join(LETTERS[x] for x in s) for s in strings]
    if contain_wildcards(strings):
        sub_name = f'submission_wildcards_{"_".join(str(len(_)) for _ in strings)}.csv'
    else:
        sub_name = f'submission_no_wildcards_{"_".join(str(len(_)) for _ in strings)}.csv'
    sub.to_csv(sub_name, index=False)
    return sub_name

tour_files = glob.glob('output_tour_*.txt') + glob.glob('best_tour_*.txt')
print("=" * 70)
for f in tour_files:
    strings = read_strings(f)
    strings.sort(key=len, reverse=True)
    print(f'File {f}, strings lenghts are {[len(s) for s in strings]}.')    
    if check_solution(strings):
        print(f'The solution is written to {write_submission_csv(strings)}')
    else:
        print('The solution is invalid.')
    print("=" * 70)


# Wildcards Optimization

We'll use the code from the [notebook](https://www.kaggle.com/yosshi999/wildcard-postprocessing-using-dynamic-programming) created by [Yosshi999](https://www.kaggle.com/yosshi999) to improve found solutions with wildcards.

In [ ]:
import itertools
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F


perms = list(map(lambda p: "".join(p), itertools.permutations("1234567")))
perm2id = {p: i for i, p in enumerate(perms)}
perms_arr = np.array([list(map(int, p)) for p in perms])

perms_onehot = np.eye(7)[perms_arr-1, :].transpose(0, 2, 1)
assert np.allclose(perms_onehot[:,0,:].astype(np.int64), (perms_arr == 1).astype(np.int64))

# print("onehot 1234567:")
# print(perms_onehot[perm2id["1234567"]])

# print("onehot 5671234:")
# print(perms_onehot[perm2id["5671234"]])

# print("correlate between 1234567 and 5671234")
left = perms_onehot[perm2id["1234567"]]
right = perms_onehot[perm2id["5671234"]]
matches = F.conv2d(
    F.pad(torch.Tensor(left[None, None, :, :]), (7, 7)),
    torch.Tensor(right[None, None, :, :]),
    padding="valid"
).numpy().reshape(-1)
# print(matches)
must_match_left2right = np.array([-1, -1, -1, -1, -1, -1, -1, 7, 6, 5, 4, 3, 2, 1, 0])
must_match_right2left = np.array([0, 1, 2, 3, 4, 5, 6, 7, -1, -1, -1, -1, -1, -1, -1])
cost_ifmatch = np.array([7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7])
# print("cost of 1234567 -> 5671234:", min(cost_ifmatch[np.equal(must_match_left2right, matches)]))
# print("cost of 5671234 -> 1234567:", min(cost_ifmatch[np.equal(must_match_right2left, matches)]))

M = F.conv2d(
    F.pad(torch.Tensor(perms_onehot[:, None, :, :]), (7, 7)),
    torch.Tensor(perms_onehot[:, None, :, :]),
    padding="valid"
).squeeze().numpy()

must_match_left2right = np.array([-1, -1, -1, -1, -1, -1, -1, 7, 6, 5, 4, 3, 2, 1, 0])
must_match_left2right_wild = np.array([-1, -1, -1, -1, -1, -1, -1, 6, 5, 4, 3, 2, 1, 0, 0])

cost_ifmatch = np.array([7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7])

costMat = np.where(M == must_match_left2right, cost_ifmatch, np.inf).min(axis=-1).astype(np.int8)
costMatWild = np.minimum(costMat, np.where(M == must_match_left2right_wild, cost_ifmatch, np.inf).min(axis=-1)).astype(np.int8)

def optimize_wildcards(words):
    found_perms = find_strings_perms(words)
    balanced_perms = rebalance_perms(found_perms)
    balanced_perms = [[''.join(str(_) for _ in perm) for perm in perms] for perms in balanced_perms]
    nodes_list = []
    table_list = []
    for i in range(3):
        word = words[i]
        nodes = [perm2id[p] for p in balanced_perms[i]]

        table = np.zeros((len(nodes), 10), np.int64)
        table[0, :] = 7
        for i in range(1, len(nodes)):
            e = costMat[nodes[i-1], nodes[i]]
            ew = costMatWild[nodes[i-1], nodes[i]]
            table[i,0] = table[i-1,0] + e
            table[i,1] = min(table[i-1,1] + e, table[i-1,0] + ew)
            table[i,2] = min(table[i-1,2], table[i-1,1]) + e # TODO: better transition
            table[i,3] = min(table[i-1,3], table[i-1,2]) + e
            table[i,4] = min(table[i-1,4], table[i-1,3]) + e
            table[i,5] = min(table[i-1,5], table[i-1,4]) + e
            table[i,6] = min(table[i-1,6], table[i-1,5]) + e
            table[i,7] = min(table[i-1,7], table[i-1,6]) + e
            table[i,8] = min(table[i-1,8], table[i-1,7]) + e
            table[i,9] = min(table[i-1,9] + e, table[i-1,8] + ew)
#         print(table[-1].min(), table[-1])
        nodes_list.append(nodes)
        table_list.append(table)

    # backtrack
    new_words = []
    wilds = []
    for nodes, table in zip(nodes_list, table_list):
        ns = [perms[nodes[-1]]]
        track = np.argmin(table[-1])
        wild = []
        for i in range(len(nodes)-2, -1, -1):
            e = costMat[nodes[i], nodes[i+1]]
            ew = costMatWild[nodes[i], nodes[i+1]]
            if track == 0:
                ns.append(perms[nodes[i]][:e])
            elif track == 1:
                if table[i, 1] + e < table[i, 0] + ew:
                    ns.append(perms[nodes[i]][:e])
                else:
                    left = np.array(list(map(int, perms[nodes[i]][ew:])))
                    right = np.array(list(map(int, perms[nodes[i+1]][:-ew])))
                    mis = np.where(left != right)[0][0]
                    wild.append(table[i, track-1]-7+ew+mis)
                    ns.append(perms[nodes[i]][:ew])
                    track = track - 1
            elif 2 <= track <= 8:
                if table[i, track] >= table[i, track-1]:
                    track = track - 1
                ns.append(perms[nodes[i]][:e])
            elif track == 9:
                if table[i, 9] + e < table[i, 8] + ew:
                    ns.append(perms[nodes[i]][:e])
                else:
                    ns.append(perms[nodes[i]][:ew])
                    left = np.array(list(map(int, perms[nodes[i]][ew:])))
                    right = np.array(list(map(int, perms[nodes[i+1]][:-ew])))
                    mis = np.where(left != right)[0][0]
                    wild.append(table[i, track-1]-7+ew+mis)
                    track = track - 1
            else:
                assert False
        assert track == 0
        wilds.append(wild)
        nsw = list("".join(ns[::-1]))
        for w in wild:
            nsw[w] = "8"
        new_words.append("".join(nsw))
    return new_words

In [ ]:
tour_files = glob.glob('submission_no_wildcards_*.csv')
print("=" * 71)
for f in tour_files:
    schedule = pd.read_csv(f).schedule.tolist()
    strings = [[INV_LETTERS[c] for c in s] for s in schedule]
    strings.sort(key=len, reverse=True)
    new_strings = optimize_wildcards(strings)
    new_strings = [[int(c) for c in s] for s in new_strings]
    new_strings.sort(key=len, reverse=True)
    print(f'File {f}.')
    print(f'Improved strings lengths from {[len(s) for s in strings]} to {[len(s) for s in new_strings]}.')
    print(f'The solution is written to {write_submission_csv(new_strings)}')
    print("=" * 71)

That's it, thank you for reading, please upvote if you find it useful.